### PyTorch 허브

- https://docs.ultralytics.com/ko/yolov5/tutorials/pytorch_hub_model_loading/

In [1]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### PyTorch 허브에 YOLOv5 로드

In [2]:
import torch

In [3]:
# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

# Image
im = "https://ultralytics.com/images/zidane.jpg"

# Inference
results = model(im)

results.pandas().xyxy[0]

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-2 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 47.7MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


,xmin,ymin,xmax,ymax,confidence,class,name
0,745.578735,48.470215,1142.694336,720.000000,0.868910,0,person
1,124.744019,197.334473,844.397583,716.650635,0.630325,0,person
2,441.238739,439.350616,498.380768,708.570923,0.616792,27,tie
3,594.081787,377.300354,635.423950,437.147827,0.274014,67,cell phone


#### 자세한 예시

In [4]:
import cv2
import torch
from PIL import Image

In [5]:
# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

# Images
for f in "zidane.jpg", "bus.jpg":
    torch.hub.download_url_to_file("https://ultralytics.com/images/" + f, f)  # download 2 images
im1 = Image.open("zidane.jpg")  # PIL image
im2 = cv2.imread("bus.jpg")[..., ::-1]  # OpenCV image (BGR to RGB)

# Inference
results = model([im1, im2], size=640)  # batch of images

# Results
results.print()
results.save()  # or .show()

results.xyxy[0]  # im1 predictions (tensor)
results.pandas().xyxy[0]  # im1 predictions (pandas)

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-2 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
100%|██████████| 49.2k/49.2k [00:00<00:00, 3.73MB/s]
100%|██████████| 134k/134k [00:00<00:00, 4.76MB/s]
C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/2: 720x1280 2 persons, 1 tie, 1 cell phone
image 2/2: 1080x810 4 persons, 1 bus
Speed: 8.5ms pre-process, 34.8ms inference, 108.1ms NMS per image at shape (2, 3, 640, 640)
Saved 2 images to runs\detect\exp


,xmin,ymin,xmax,ymax,confidence,class,name
0,744.907715,47.730957,1142.076172,716.559448,0.867366,0,person
1,127.860870,197.634583,845.973877,710.378113,0.637458,0,person
2,441.277222,439.579651,498.367310,708.055603,0.635027,27,tie
3,594.006104,376.740356,635.635010,437.111206,0.277625,67,cell phone


#### 추론 설정

YOLOv5 모델에는 신뢰도 임계값, IoU 임계값 등과 같은 다양한 추론 속성이 포함되어 있으며, 이를 설정할 수 있습니다

In [6]:
model.conf = 0.25  # NMS confidence threshold
iou = 0.45  # NMS IoU threshold
agnostic = False  # NMS class-agnostic
multi_label = False  # NMS multiple labels per box
classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
max_det = 1000  # maximum number of detections per image
amp = False  # Automatic Mixed Precision (AMP) inference

results = model(im, size=320)  # custom inference size

C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


#### 장치

모델은 생성 후 모든 디바이스로 전송할 수 있습니다

In [8]:
model.cpu()  # CPU
model.cuda()  # GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model.to(device)  # i.e. device=torch.device(0)

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [9]:
device

device(type='cuda', index=0)

모델을 직접 만들 수도 있습니다. device

In [11]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s", device=device)  # load on CPU. device=device or device='cpu'

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-2 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


#### 무음 출력

다음을 사용하여 모델을 자동으로 로드할 수 있습니다. `_verbose=False`

In [12]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s", _verbose=False)  # load silently

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-2 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


#### 입력 채널

기본 3개가 아닌 4개의 입력 채널로 사전 학습된 YOLOv5s 모델을 로드합니다

In [13]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s", channels=4)

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-2 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)


                 from  n    params  module                                  arguments                     
  0                -1  1      4672  models.common.Conv                      [4, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3         

#### 클래스 수

기본값 80이 아닌 10개의 출력 클래스로 사전 학습된 YOLOv5s 모델을 로드합니다

In [14]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s", classes=10)

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-2 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Overriding model.yaml nc=80 with nc=10

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1 

#### 강제 리로드

위의 단계에서 문제가 발생하면 설정 `force_reload=True` 기존 캐시를 삭제하고 PyTorch 허브에서 최신 YOLOv5 버전을 강제로 새로 다운로드하면 도움이 될 수 있습니다.

In [15]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s", force_reload=True)  # force reload

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\perso/.cache\torch\hub\master.zip
YOLOv5  2025-1-28 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


#### 스크린샷 추론

데스크톱 화면에서 추론을 실행하려면 다음과 같이 하세요

In [16]:
import torch
from PIL import ImageGrab

# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

# Image
im = ImageGrab.grab()  # take a screenshot

# Inference
results = model(im)

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-28 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


#### 다중GPU 추론

YOLOv5 모델을 스레드 추론을 통해 여러 GPU에 병렬로 로드할 수 있습니다

In [18]:
import threading

import torch


def run(model, im):
    """Performs inference on an image using a given model and saves the output; model must support `.save()` method."""
    results = model(im)
    results.save()


# Models
model0 = torch.hub.load("ultralytics/yolov5", "yolov5s", device=device)
model1 = torch.hub.load("ultralytics/yolov5", "yolov5s", device=device)

# Inference
threading.Thread(target=run, args=[model0, "https://ultralytics.com/images/zidane.jpg"], daemon=True).start()
threading.Thread(target=run, args=[model1, "https://ultralytics.com/images/bus.jpg"], daemon=True).start()

Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-28 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
Using cache found in C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-1-28 Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 with Max-Q Design, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\perso/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Saved 1 image to runs\detect\exp4
Saved 1 image to runs\detect\exp5


- 이전보다 퍼센트가 올라감